<a href="https://colab.research.google.com/github/bigirimanainnocent12/ANOVA/blob/main/ANOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Analyse de la variance à un facteur***

# ***Introduction***

Rémi est un philatéliste passionné. Il collectionne particulièrement les timbres d'Allemagne,d'Autriche, de Belgique et de France. Il a l'impression qu'il a plus de mal à décoller sans les abîmer les timbres français. Il pense que ces timbres sont peut-être moins épais que les autres. Il sélectionne de manière aléatoire des timbres non abîmés dans sa collection en se limitant aux 4 pays dont il a le plus de timbres. Il fait mesurer l'épaisseur de ces timbres par une société équipée pour faire des mesures en micromètre. Il obtient le jeu de données timbres.csv. Calculez les statistiques descriptives puis faites le traitement statistique adéquat pour que Rémi puisse déterminer si l'épaisseur moyenne des timbres diffère selon les pays.


In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ***importation de la base de données***

In [ ]:
df=pd.read_csv('timbres.csv',sep=';')
df.head()

,Unnamed: 0,epaisseur,pays
0,1,263,Belgique
1,2,259,Belgique
2,3,246,Belgique
3,4,255,Belgique
4,5,260,Belgique


# ***Statistiques Descriptives***

In [ ]:
df.groupby('pays')['epaisseur'].describe()

,count,mean,std,min,25%,50%,75%,max
pays,,,,,,,,
Allemagne,19.0,251.631579,6.048053,238.0,246.50,252.0,256.0,261.0
Autriche,25.0,251.840000,6.276145,242.0,246.00,252.0,258.0,265.0
Belgique,23.0,253.217391,6.828793,237.0,247.50,254.0,258.0,265.0
France,22.0,210.772727,8.518505,196.0,205.25,211.0,216.0,230.0


# ***Les hypothèses du test et choix du test***


La question de Rémy est de savoir si l'épaisseur moyenne des timbres diffère selon les pays. Nous allons utiliser un test paramétrique, tel que l'analyse de la variance (ANOVA), pour tenter de répondre à cette question.

Le modèle s'écrit en général:

 $Y_{ij}=\mu+\alpha_j+\epsilon_{ij}$ avec ∀ j ∈ {1,2,3,4}, $\{Y_{1j} $,...,
 $Y_{n_{j}j} \}$, les réponses des observations pour lesquelles x = j et ∀ i ∈ {1,..,$n_{j}$}, $\mathbb{E}(\epsilon_{ij})=0$


***Les hypothèses du test sont: ***

- $H_0$: Il n'y a pas de différence significative entre les
moyennes des épaisseurs des timbres des différents pays.
- $H_1$ : Il y a au moins une moyenne des timbres
différente des autres moyennes des épaisseurs des timbres des différents pays.








# ***Choix du seuil***

Le seuil de significativité ($\alpha$) est généralement fixé à 0,05. Cela signifie que si la p-valeur obtenue à partir du test est inférieure à 0,05, nous pouvons rejeter l'hypothèse nulle et conclure qu'il existe une différence significative entre les moyennes des épaisseurs des timbres des différents pays.


# ***Vérifications des conditions d’application du test (ANOVA)***

# **1. Test de Shapiro- Wilk**

Le test de Shapiro-Wlk est un test statistique utilisé pour tester si un échantillon de données proviennent d'une population normalement distribution.
Les hypothèses du test sont:

$H_0$: l'échantillon est issu d'une population normalement distribuée

$H_1$: l'échantillon n'est pas issu d'une population normalement distribuée

**Résulats du test:**

In [ ]:
from scipy import stats
resultats_shapiro = df.groupby('pays')['epaisseur'].apply(lambda x: stats.shapiro(x))

for groupe, resultat in resultats_shapiro.items():
    print(f"Groupe {groupe} : Statistique W = {resultat.statistic:.4f}, p-valeur = {resultat.pvalue:.4f}")

Groupe Allemagne : Statistique W = 0.9658, p-valeur = 0.6912
Groupe Autriche : Statistique W = 0.9516, p-valeur = 0.2723
Groupe Belgique : Statistique W = 0.9751, p-valeur = 0.8086
Groupe France : Statistique W = 0.9798, p-valeur = 0.9127


Toutes les p-valeurs sont supérieures à 0,05. Nous acceptons donc $H_0$ et concluons que les trois échantillons proviennent de populations normalement distribuées.

# **Test de Bartlett**

Le test de Bartlett est un test statistique utilisé pour vérifier l'homogeneité des variances de plus de deux échantillons.

**Les hypothèses test sont:**

- $H_0$: Il n'y a pas de différence significative entre les
variances des épaisseurs des timbres des différents pays.

- $H_1$ : Il y a au moins une variance des timbres
différente des autres variances des épaisseurs des timbres des différents pays.

In [ ]:
from scipy import stats
groupes = [pays['epaisseur'].values for nom, pays in df.groupby('pays')]


# Exécution du test de Bartlett
stat, p_value = stats.bartlett(*groupes)
print(f"Statistique de Bartlett = {stat:.4f}, p-valeur = {p_value:.4f}")

Statistique de Bartlett = 3.0509, p-valeur = 0.3838


La p-valeurs=0.3838 supérieures à 0,05. Nous acceptons donc $H_0$ et concluons que les trois échantillons ont une même  variabilité (variances égales).

# ***Résultats du test ANOVA: ***

In [ ]:
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

modele=ols('epaisseur ~ C(pays)',data=df).fit()
table=sm.stats.anova_lm(modele,typ=2)
print(table)


                sum_sq    df           F        PR(>F)
C(pays)   28530.442268   3.0  194.619308  5.824759e-38
Residual   4153.557732  85.0         NaN           NaN


La valeur de la statistique est égale à 194,619308. Sous $H_0$, elle suit une loi de Fisher avec 3 et 85 degrés de liberté. La p-valeur  est largement inférieure à 0,05.N ous rejetons $H_0$ et nous concluons qu’il y a au moins une moyenne d’épaisseur de timbres différente des autres moyennes d’épaisseur des timbres des différents pays.

Nous pouvons procéder à des tests post-hoc pour identifier quels groupes diffèrent significativement les uns des autres. Parmi les tests post-hoc couramment utilisés, on retrouve le test de Tukey, le test de Bonferroni et le test de Scheffé.

# **Test de comparaison multiples**

Les tests de comparaisons multiples avec la correction de Tukey est une méthode d'analyse statistique permettant de comparer tous les groupes entre eux de manière simultanée. Cette méthode est utilisée lorsque nous avons plus de deux groupes à comparer et que nous souhaitons déterminer les différences significatives entre ces groupes.

In [49]:
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

test=df.filter(items=['epaisseur','pays'])
tukey=pairwise_tukeyhsd(endog=test['epaisseur'],groups=test['pays'],alpha=0.05)
print(tukey)

    Multiple Comparison of Means - Tukey HSD, FWER=0.05    
  group1   group2  meandiff p-adj   lower    upper   reject
-----------------------------------------------------------
Allemagne Autriche   0.2084 0.9997   -5.367   5.7839  False
Allemagne Belgique   1.5858 0.8841  -4.0933    7.265  False
Allemagne   France -40.8589    0.0 -46.5961 -35.1216   True
 Autriche Belgique   1.3774 0.9037  -3.9154   6.6702  False
 Autriche   France -41.0673    0.0 -46.4224 -35.7122   True
 Belgique   France -42.4447    0.0 -47.9077 -36.9817   True
-----------------------------------------------------------


D’après le tableau ci-dessus, les résultats montrent qu'il y a une différence significative dans les moyennes des timbres entre l'Allemagne et la France, l'Autriche et la France, ainsi que la Belgique et la France. Les autres comparaisons entre les pays n'ont pas montré de différences significatives. Cela signifie que la moyenne des timbres en France est significativement différente de celles de l'Allemagne, de l'Autriche et de la Belgique. Donc les timbres français sont assez épais par rapport autres pays.